## **Install Selenium**

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.E12CxDrbuC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.PenLhXwQNp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ax5My6TcB2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

## **Import libraries**

In [ ]:
import pandas as pd
import os
import requests
import time
import re
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## **Crawl poem**

In [ ]:
WEBDRIVER_DELAY_TIME_INT = 10
TIMEOUT_INT = 10
service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(TIMEOUT_INT)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [ ]:
def clean_poem_html(html):
    html = re.sub(r'<img.*?>', '', html, flags=re.IGNORECASE)
    html = re.sub(r'<i\b[^>]*>.*?</i>', '', html, flags=re.IGNORECASE | re.DOTALL)
    html = re.sub(r'</?b>', '', html, flags=re.IGNORECASE)
    html = re.sub(r'<span.*?</span>', '', html, flags=re.DOTALL)

    html = re.sub(r'</?p>', '\n', html, flags=re.IGNORECASE)

    html = re.sub(r'<br\s*/?>', '\n', html, flags=re.IGNORECASE)

    html = re.sub(r'\n{2,}', '\n', html) 
    html = re.sub(r' +', ' ', html)  

    return html.strip()

In [ ]:
def format_luc_bat(text):
    lines = text.split('\n')
    formatted_lines = []

    for line in lines:
        line = line.strip()
        if line:
            words = line.split()
            if len(words) >= 6:  
                formatted_lines.append(" ".join(words))

    return "\n".join(formatted_lines)

In [ ]:
def process_poem_content(html, poem_src, poem_url, default_title=''):
    cleaned = clean_poem_html(html)

    title_match = re.search(r'<h1>(.*?)</h1>', html, flags=re.IGNORECASE)
    title = title_match.group(1).strip() if title_match else default_title

    content = format_luc_bat(cleaned)

    return [{
        'title': title,
        'content': content,
        'source': poem_src,
        'url': poem_url
    }]


In [ ]:
def extract_poem_source(driver):
    try:
        poem_src_tag = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="small"]'))
        )
        return poem_src_tag.text
    except Exception:
        return ''

In [ ]:
def extract_poem_links(driver, page_idx):
    main_url = f'https://www.thivien.net/searchpoem.php?VNPoem=th%C3%B4i&PoemType=13&ViewType=1&Country=2&Sort=Views&SortOrder=desc&Page={page_idx}'
    driver.get(main_url)
    time.sleep(random.uniform(3, 5))

    content_tags_xpath = '//*[@class="page-content container"]//div[@class="page-content-main"]//div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    poem_links = []
    for tag in content_tags:
        try:
            link_element = tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            poem_title = link_element.text
            poem_url = link_element.get_attribute('href')
            poem_links.append({'title': poem_title, 'url': poem_url})
        except Exception as e:
            print(f"Error extracting link: {e}")
            continue
    return poem_links

In [ ]:
def check_page_structure(driver, page_idx):
    main_url = f'https://www.thivien.net/searchpoem.php?VNPoem=th%C3%B4i&PoemType=13&ViewType=1&Country=2&Sort=Views&SortOrder=desc&Page={page_idx}'
    driver.get(main_url)
    time.sleep(random.uniform(3, 5))  # Wait for the page to load

    # Find all "list-item" divs
    content_tags_xpath = '//*[@class="page-content container"]//div[@class="page-content-main"]//div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    print(f"Page {page_idx}: Found {len(content_tags)} list-item elements.")

    missing_titles_count = 0
    for idx, tag in enumerate(content_tags):
        try:
            # Attempt to find the <h4> with class "list-item-header"
            link_element = tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            poem_title = link_element.text
            poem_url = link_element.get_attribute('href')
            print(f"  Item #{idx+1} => Title: {poem_title}, URL: {poem_url}")
        except Exception as e:
            missing_titles_count += 1
            # Print debug info: outerHTML of the failing element
            print(f"  ERROR in item #{idx+1} => {e}")
            print("  Outer HTML:")
            print(tag.get_attribute("outerHTML"))
            print("-"*50)

    if missing_titles_count == 0:
        print(f"All items on page {page_idx} have valid <h4 class=\"list-item-header\">.")
    else:
        print(f"{missing_titles_count} items on page {page_idx} did NOT have <h4 class=\"list-item-header\">.")
    print("="*70)


def debug_structure(driver, start_page=1, end_page=2):
    """
    Runs the debug structure check for pages from start_page to end_page.
    """
    for page_idx in range(start_page, end_page+1):
        check_page_structure(driver, page_idx)

In [ ]:
check_page_structure(driver, page_idx=1)

Page 1: Found 10 list-item elements.
  Item #1 => Title: Việt Bắc, URL: https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Vi%E1%BB%87t-B%E1%BA%AFc/poem-v9h1P_8l-sRaiDRUqg2JRQ
  Item #2 => Title: Khi con tu hú, URL: https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Khi-con-tu-h%C3%BA/poem-nUKqIuV4aZXcoO9rCpAkhQ
  Item #3 => Title: Tiếng ru, URL: https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Ti%E1%BA%BFng-ru/poem-k-qlGdQk1sf_Au3xAiyXoQ
  Item #4 => Title: Lịch sử nước ta, URL: https://www.thivien.net/H%E1%BB%93-Ch%C3%AD-Minh/L%E1%BB%8Bch-s%E1%BB%AD-n%C6%B0%E1%BB%9Bc-ta/poem-NjHZhJYtu1u28vSkEmRsug
  Item #5 => Title: Lỡ bước sang ngang, URL: https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/L%E1%BB%A1-b%C6%B0%E1%BB%9Bc-sang-ngang/poem-LkgRymP1ghPPuP-xPE0EkA
  Item #6 => Title: Người hàng xóm, URL: https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/Ng%C6%B0%E1%BB%9Di-h%C3%A0ng-x%C3%B3m/poem-S3uBh8NU6oIs819c8VePRg
  Item #7 => Title: Hồi 03: Vân Tiên cứu Kiều Nguyệt Nga, URL: https://www.thivien.ne

In [ ]:
def scrape_poem(driver, poem_url, poem_title):
    driver.get(poem_url)
    time.sleep(random.uniform(3, 5))
    poem_content_tag = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.poem-content'))
    )
    html_content = poem_content_tag.get_attribute('innerHTML')
    poem_src = extract_poem_source(driver)
    return process_poem_content(html_content, poem_src, poem_url, default_title=poem_title)

def scrape_poems(driver, num_pages=10):
    datasets = []
    for page_idx in tqdm(range(1, num_pages + 1)):
        poem_links = extract_poem_links(driver, page_idx)
        print(poem_links)
        for poem in poem_links:
            poem_url = poem['url']
            poem_title = poem['title']
            try:
                # Pass poem_title  scrape_poem
                poems = scrape_poem(driver, poem_url, poem_title)
                datasets.extend(poems)
            except Exception as e:
                print(f"Error processing {poem_url}: {e}")
                continue
    return datasets

In [ ]:
# Crawl poems from 10 pages
datasets = scrape_poems(driver, num_pages=10)
driver.quit()

  0%|          | 0/10 [00:00<?, ?it/s]

[{'title': 'Việt Bắc', 'url': 'https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Vi%E1%BB%87t-B%E1%BA%AFc/poem-v9h1P_8l-sRaiDRUqg2JRQ'}, {'title': 'Khi con tu hú', 'url': 'https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Khi-con-tu-h%C3%BA/poem-nUKqIuV4aZXcoO9rCpAkhQ'}, {'title': 'Tiếng ru', 'url': 'https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu/Ti%E1%BA%BFng-ru/poem-k-qlGdQk1sf_Au3xAiyXoQ'}, {'title': 'Lịch sử nước ta', 'url': 'https://www.thivien.net/H%E1%BB%93-Ch%C3%AD-Minh/L%E1%BB%8Bch-s%E1%BB%AD-n%C6%B0%E1%BB%9Bc-ta/poem-NjHZhJYtu1u28vSkEmRsug'}, {'title': 'Lỡ bước sang ngang', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/L%E1%BB%A1-b%C6%B0%E1%BB%9Bc-sang-ngang/poem-LkgRymP1ghPPuP-xPE0EkA'}, {'title': 'Người hàng xóm', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/Ng%C6%B0%E1%BB%9Di-h%C3%A0ng-x%C3%B3m/poem-S3uBh8NU6oIs819c8VePRg'}, {'title': 'Hồi 03: Vân Tiên cứu Kiều Nguyệt Nga', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-%C4%90%C3%ACnh-Chi%E1%BB%83u/H%E1%BB

 10%|█         | 1/10 [01:30<13:30, 90.07s/it]

[{'title': 'Tiếng hát mùa gặt', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-Duy/Ti%E1%BA%BFng-h%C3%A1t-m%C3%B9a-g%E1%BA%B7t/poem-Zt-XpsEc1Aqf2Osp3F0lYA'}, {'title': 'Tâm sự nàng Thuý Vân', 'url': 'https://www.thivien.net/Tr%C6%B0%C6%A1ng-Nam-H%C6%B0%C6%A1ng/T%C3%A2m-s%E1%BB%B1-n%C3%A0ng-Thu%C3%BD-V%C3%A2n/poem-qK0dpfYL9uuGLe2cq7-64A'}, {'title': 'Đi đâu mà vội mà vàng', 'url': 'https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi%E1%BB%87t-Nam/%C4%90i-%C4%91%C3%A2u-m%C3%A0-v%E1%BB%99i-m%C3%A0-v%C3%A0ng/poem-jX-ZCwoG9eELsZgK9ZCz4g'}, {'title': 'Nghĩ về Tấm Cám', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-Ng%E1%BB%8Dc-H%C6%B0ng/Ngh%C4%A9-v%E1%BB%81-T%E1%BA%A5m-C%C3%A1m/poem-7DGh9AyxGxvrsSyZMppyMA'}, {'title': 'Con đường', 'url': 'https://www.thivien.net/Phan-Th%E1%BB%8B-Thanh-Nh%C3%A0n/Con-%C4%91%C6%B0%E1%BB%9Dng/poem-KAziFPjbbO4e3z6UU1eeDw'}, {'title': 'Em về Tam Đảo', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3m-Th%E1%BA%AFng/Em-v%E1%BB%81-Tam-%C4%90%E1%BA%A3o/poem-IKcgP3_7

 20%|██        | 2/10 [03:09<12:45, 95.70s/it]

...

 80%|████████  | 8/10 [12:49<03:07, 93.51s/it] 

[{'title': 'Gừng và muối', 'url': 'https://www.thivien.net/M%C6%B0%E1%BB%9Dng-M%C3%A1n/G%E1%BB%ABng-v%C3%A0-mu%E1%BB%91i/poem-V4veYGGGdWUfhsLWeBPnkQ'}, {'title': 'Nhầm', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/Nh%E1%BA%A7m/poem-CMstvt7bibwSCtUvZQrLnQ'}, {'title': 'Mưa', 'url': 'https://www.thivien.net/Xu%C3%A2n-Di%E1%BB%87u/M%C6%B0a/poem-ICDGieRndD99QHqHs3vJgA'}, {'title': 'Hồi 12: Kiều gặp Thúc Sinh', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-Du/H%E1%BB%93i-12-Ki%E1%BB%81u-g%E1%BA%B7p-Th%C3%BAc-Sinh/poem-W2wLElG7-g_l8WOLalgL0Q'}, {'title': 'Chuồn chuồn', 'url': 'https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi%E1%BB%87t-Nam/Chu%E1%BB%93n-chu%E1%BB%93n/poem-UUgEfdUkjts8zuKJypK4eg'}, {'title': 'Lời ru', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3m-Th%E1%BA%AFng/L%E1%BB%9Di-ru/poem-QCCTPBeFE0aDFtcbTEyzdQ'}, {'title': 'Uống rượu', 'url': 'https://www.thivien.net/Tr%E1%BA%A7n-Huy%E1%BB%81n-Tr%C3%A2n/U%E1%BB%91ng-r%C6%B0%E1%BB%A3u/poem-80S6ruOi7Ouc08yZYWcFjQ'}

 90%|█████████ | 9/10 [14:30<01:35, 95.72s/it]

[{'title': 'Nói tào lao', 'url': 'https://www.thivien.net/Xu%C3%A2n-Di%E1%BB%87u/N%C3%B3i-t%C3%A0o-lao/poem-OwEmH3bLJZNit-4bESiOsg'}, {'title': 'Hồi 17: Nguyệt Nga nhảy xuống sông, nhờ Quan Âm đem vô bờ; vào vườn gặp Bùi ông đem về nuôi, bị Bùi Kiệm dỗ ngon dỗ ngọt', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-%C4%90%C3%ACnh-Chi%E1%BB%83u/H%E1%BB%93i-17-Nguy%E1%BB%87t-Nga-nh%E1%BA%A3y-xu%E1%BB%91ng-s%C3%B4ng-nh%E1%BB%9D-Quan-%C3%82m-%C4%91em-v%C3%B4-b%E1%BB%9D-v%C3%A0o-v%C6%B0%E1%BB%9Dn-g%E1%BA%B7p-B%C3%B9i-%C3%B4ng-%C4%91em-v%E1%BB%81-nu%C3%B4i-b%E1%BB%8B-B%C3%B9i-Ki%E1%BB%87m/poem-6txf0DyAAR2Z-mUTvv748g'}, {'title': 'Hồi 21: Kim Trọng tìm Kiều', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-Du/H%E1%BB%93i-21-Kim-Tr%E1%BB%8Dng-t%C3%ACm-Ki%E1%BB%81u/poem-2l0CsjQ9tANWPMDBtirmRg'}, {'title': 'Đêm cuối cùng', 'url': 'https://www.thivien.net/Nguy%E1%BB%85n-B%C3%ADnh/%C4%90%C3%AAm-cu%E1%BB%91i-c%C3%B9ng/poem-ZjSu27nDCKsyKWbOwBvzWw'}, {'title': 'Đêm chiêm bao', 'url': 'https://www.thivie

100%|██████████| 10/10 [16:33<00:00, 99.38s/it] 


## **Save data to file**

In [ ]:
df = pd.DataFrame(datasets)
df.to_csv('poem68_dataset.csv', index=True)

In [ ]:
df

,title,content,source,url
0,Việt Bắc,Tiếng ai tha thiết bên cồn\nBâng khuâng trong ...,10-1954\n\nChiến dịch Điện Biên Phủ kết thúc t...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
1,Khi con tu hú,"Khi con tu hú gọi bầy\nLúa chiêm đang chín, tr...","Huế, tháng 7-1939\n\nBài thơ Khi con tu hú đượ...",https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
2,Tiếng ru,"Con ong làm mật, yêu hoa\nCon cá bơi, yêu nước...",Ba khổ thơ đầu bài này được sử dụng trong SGK ...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
3,Lịch sử nước ta,"Dân ta phải biết sử ta,\nCho tường gốc tích nư...",Đầu năm 1942\n\nĐể giáo dục tinh thần yêu nước...,https://www.thivien.net/H%E1%BB%93-Ch%C3%AD-Mi...
4,Lỡ bước sang ngang,"“- Em ơi, em ở lại nhà,\nVườn dâu em đốn, mẹ g...",Bài thơ này được đăng lần đầu trên Tiểu thuyết...,https://www.thivien.net/Nguy%E1%BB%85n-B%C3%AD...
...,...,...,...,...
95,Lạ,"Người nào xa lạ đâu đây\nNgười mi thanh sáng, ...",,https://www.thivien.net/Xu%C3%A2n-Di%E1%BB%87u...
96,Hồi 09: Kiều rơi vào tay Tú Bà,Rèm trong đã thấy một người bước ra.\nĂn cao l...,Đoạn này ứng với Hồi 8 trong nguyên truyện: “V...,https://www.thivien.net/Nguy%E1%BB%85n-Du/H%E1...
97,Áo đỏ em đi trong chiều tuyết trắng,Tất cả đã trở về màu xám\nChỉ còn em và tuyết ...,12.1987\n\nBài thơ đề tặng chị Phan Bích Thiện...,https://www.thivien.net/Ph%E1%BA%A1m-Ti%E1%BA%...
98,Hồi 01,"1. Trăm năm một sợi chỉ hồng,\nBuộc người tài ...",,https://www.thivien.net/Nguy%E1%BB%85n-Huy-T%E...


In [ ]:
print(df['title'][5])
print(df['content'][5])

Người hàng xóm
Nhà nàng ở cạnh nhà tôi,
Cách nhau cái giậu mùng tơi xanh rờn.
Hai người sống giữa cô đơn,
Nàng như cũng có nỗi buồn giống tôi.
Giá đừng có giậu mùng tơi,
Thế nào tôi cũng sang chơi thăm nàng.
Tôi chiêm bao rất nhẹ nhàng...
Có con bướm trắng thường sang bên này.
Bướm ơi! Bướm hãy vào đây!
Cho tôi hỏi nhỏ câu này chút thôi...
Chả bao giờ thấy nàng cười,
Nàng hong tơ ướt ra ngoài mái hiên,
Mắt nàng đăm đắm trông lên...
Con bươm bướm trắng về bên ấy rồi!
Bỗng dưng tôi thấy bồi hồi,
Tôi buồn tự hỏi: “Hay tôi yêu nàng?”
- Không, từ ân ái nhỡ nhàng,
Tình tôi than lạnh gio tàn làm sao!
Tơ hong nàng chả cất vào,
Con bươm bướm trắng hôm nào cũng sang.
Mấy hôm nay chẳng thấy nàng,
Giá tôi cũng có tơ vàng mà hong.
Cái gì như thể nhớ mong?
Nhớ nàng? Không! Quyết là không nhớ nàng!
Vâng, từ ân ái nhỡ nhàng,
Lòng tôi riêng nhớ bạn vàng ngày xưa.
Tầm tầm giời cứ đổ mưa,
Hết hôm nay nữa là vừa bốn hôm.
Cô đơn buồn lại thêm buồn...
Tạnh mưa bươm bướm biết còn sang chơi?
Hôm nay mưa đã tạ